# Exercício Busca A*
Represente a operação da busca A* aplicada ao problema de ir até Bucareste a partir
de Arad usando a heurística de distância em linha reta. Isto é, mostre a sequência de
nós que o algoritmo irá considerar e a pontuação de f, g e h para cada nó (Faça a
implementação em uma linguagem de programação, preferencialmente em Python).


Legenda:

    .g(n) = custo até o momento para alcançar n
    .h(n) = custo estimado de n até o objetivo
    .f(n) = custo total estimado do caminho através de n até o objetivo.

Para começar nosso problema, vamos utilizar uma fila de prioridade mínima (Min Heap) para modelar cada caminho que vamos vasculhar, utilizando o custo f(n) como critério

In [6]:
import heapq
from queue import LifoQueue, Queue

class PriorityQueue:
    def __init__(self):
        self._queue = []
        self._index = 0

    def push(self, item, priority):
        heapq.heappush(self._queue, (priority, self._index, item))
        self._index += 1

    def pop(self):
        # Pop the smallest item from the queue
        return heapq.heappop(self._queue)[-1]

    def is_empty(self):
        return len(self._queue) == 0
    
    def __repr__(self):
        return str(self._queue)

A classe Vertex representa um vértice em um grafo, sendo capaz de guardar os vértices que chegaram nela através do atributo
"city_came_by" para permitir criar um caminho direcionado no grafo de forma simples, seus vizinhos e seu custo f(n)

In [7]:
class Vertex:
        def __init__(self, city, cost_function_f=0, city_came_by=None, color="branco", neighbors=None):
            self.city = city
            self.neighbors = {} if neighbors is None else neighbors
            self.color = color
            self.cost_function_f = cost_function_f
            self.city_came_by = city_came_by

        def add_neighbor(self, neighbor, distance):
            self.neighbors[neighbor] = distance

        def get_neighbors(self):
            return self.neighbors.keys()

        def get_distance_to(self, neighbor):
            return self.neighbors[neighbor]

        def __repr__(self):
            return self.city + " V"

        def is_neighbor(self, neighbor):
            return neighbor in self.neighbors

        def clone(self):
            vertex = Vertex(self.city, self.cost_function_f, self.city_came_by, self.color)
            vertex.neighbors = self.neighbors.copy()
            return vertex

Para modelar o relacionamento entre as cidades, criamos um grafo, capaz de relacionar os custos entre os vizinhos e usar a busca a*

In [8]:
class Graph:
    def __init__(self):
        self.vertices = {}
        self.heuristic = {}
        self.paths_queue = PriorityQueue()
        
    def add_vertex(self, city):
        if city not in self.vertices:
            self.vertices[city] = Vertex(city)

    def add_edge(self, start, end, distance):
        self.vertices[start].add_neighbor(self.vertices[end], distance)
        self.vertices[end].add_neighbor(self.vertices[start], distance)


    def local_cost_function(self, vertex, neighbor):
        return vertex.get_distance_to(neighbor) + self.heuristic[neighbor.city]
    
    def print_path(self, end_vertex):
        lifo_queue = LifoQueue() # Pilha de chamada
        while end_vertex is not None:
            lifo_queue.put(end_vertex.city)
            end_vertex = end_vertex.city_came_by
        print("caminho:")
        while lifo_queue.qsize() > 1:
            print(lifo_queue.get() + " - > ", end="")
        print(lifo_queue.get())
    
    def cost_function_g(self, vertex):
        if vertex.cost_function_f == 0 or vertex.city_came_by == None:
            return 0
        return vertex.cost_function_f - self.heuristic[vertex.city]
    
    def print_cost_g(self, start_vertex):
        print("custo g: " + str(self.cost_function_g(start_vertex)))
    
    def print_cost_h(self, end_vertex):
        print("custo h: " + str(self.heuristic[end_vertex.city]))

    def a_starCall(self, start_vertex, end_city):
        end = self.vertices[end_city]
        start_vertex.color = "cinza"
        
        # Verifica se ja chegou
        if start_vertex.city == end_city:
            verticeFinal = self.paths_queue.pop()
            if verticeFinal.cost_function_f >= self.vertices[end_city].cost_function_f:
                self.print_path(start_vertex)
                return 1
            
            # Se nao chegou reset
            start_vertex.color = "branco"
            start_vertex.cost_function_f = 0
            return 0

        # Se nao chegou
        
        # Remova o custo da heuristica do custo_f do vertice        
        start_vertex.cost_function_f = self.cost_function_g(start_vertex)
        
        # Passa pelos vizinhos
        for vertex in start_vertex.get_neighbors():
            if vertex.color == "branco":
                vertex_clone = vertex.clone()
                vertex_clone.city_came_by = start_vertex
                vertex_clone.cost_function_f = (
                    start_vertex.cost_function_f
                    + self.local_cost_function(start_vertex, vertex)
                )
                self.paths_queue.push(vertex_clone, vertex_clone.cost_function_f)
        return 0

    def a_star(self, start_city, end_city, heuristic):
        self.heuristic = heuristic
        self.paths_queue = PriorityQueue()
        start_vertex = self.vertices[start_city]
        self.paths_queue.push(start_vertex, 0)

        for vertex in self.vertices.values():
            vertex.color = "branco"
            
        while not self.paths_queue.is_empty():
            new_vertex = self.paths_queue.pop()
            if new_vertex.city_came_by is not None:
                print(new_vertex.city_came_by.city + " -> " + new_vertex.city)
                self.print_cost_g(new_vertex)
                self.print_cost_h(new_vertex)
                print("custo f: " + str(new_vertex.cost_function_f))
            else:
                print(new_vertex.city)
                self.print_cost_g(new_vertex)
                self.print_cost_h(new_vertex)
                print("custo f: " + str(self.heuristic[new_vertex.city]))
            if self.a_starCall(new_vertex, end_city) == 1:
                return "sucesso"
        return 0
       

Lendo o grafo passado

In [9]:
data = []

with open("Grafo.txt", "r") as file:
    for line in file:
        start, end, distance = line.strip().split(";")
        data.append((start, end, int(distance)))

print(data)

[('Oradea', 'Zerind', 71), ('Oradea', 'Sibiu', 151), ('Arad', 'Zerind', 75), ('Arad', 'Sibiu', 140), ('Arad', 'Timisoara', 118), ('Timisoara', 'Lugoj', 111), ('Lugoj', 'Mehadia', 70), ('Mehadia', 'Drobeta', 75), ('Drobeta', 'Craiova', 120), ('Craiova', 'Rimnicu Vilcea', 146), ('Craiova', 'Pitesti', 138), ('Rimnicu Vilcea', 'Pitesti', 97), ('Rimnicu Vilcea', 'Sibiu', 80), ('Sibiu', 'Fagaras', 99), ('Fagaras', 'Bucareste', 211), ('Pitesti', 'Bucareste', 101), ('Bucareste', 'Giurgiu', 90), ('Bucareste', 'Urziceni', 85), ('Urziceni', 'Hirsova', 98), ('Hirsova', 'Eforie', 86), ('Urziceni', 'Vaslui', 142), ('Vaslui', 'Iasi', 92), ('Iasi', 'Neamt', 87)]


Lendo a heurística passada

In [10]:
my_heuristic = {}

with open("Heuristica.txt", "r") as file:
    for line in file:
        city, heuristic = line.strip().split(";")
        my_heuristic[city] = int(heuristic)

print(my_heuristic)

{'Arad': 366, 'Bucareste': 0, 'Craiova': 160, 'Drobeta': 242, 'Eforie': 161, 'Fagaras': 176, 'Giurgiu': 77, 'Hirsova': 151, 'Iasi': 226, 'Lugoj': 244, 'Mehadia': 241, 'Neamt': 234, 'Oradea': 380, 'Pitesti': 100, 'Rimnicu Vilcea': 193, 'Sibiu': 253, 'Timisoara': 329, 'Urziceni': 80, 'Vaslui': 199, 'Zerind': 374}


Testando:

In [11]:
graph = Graph()

for start, end, distance in data:
    graph.add_vertex(start)
    graph.add_vertex(end)
    graph.add_edge(start, end, distance)

graph.a_star("Arad", "Bucareste", my_heuristic)

Arad
custo g: 0
custo h: 366
custo f: 366
Arad -> Sibiu
custo g: 140
custo h: 253
custo f: 393
Sibiu -> Rimnicu Vilcea
custo g: 220
custo h: 193
custo f: 413
Sibiu -> Fagaras
custo g: 239
custo h: 176
custo f: 415
Rimnicu Vilcea -> Pitesti
custo g: 317
custo h: 100
custo f: 417
Pitesti -> Bucareste
custo g: 418
custo h: 0
custo f: 418
caminho:
Arad - > Sibiu - > Rimnicu Vilcea - > Pitesti - > Bucareste


'sucesso'